In [1]:
import pandas as pd
import numpy as np
import requests

In [2]:
df = pd.read_csv('广州写字楼数据.csv')

In [3]:
def replacestr(string):
    string = str(string).replace('[','')
    string = string.replace(']','')
    string = string.replace("'",'')
    return string

df['loupandizhi'] = df['loupandizhi'].apply(replacestr)

list_location = df['loupandizhi'].value_counts().index.tolist()

dict_location = {}
Key = '6b95b84cab6100c0a6f52c62ece97e4b'
My_key = 'f654d644aeae0e1da847d7ca408f4370'
url_pre = 'https://restapi.amap.com/v3/geocode/geo?key={}&address={}&city={}'
counter = 0
for i in list_location:
    location = i.split(",")[-1].strip()
    url = url_pre.format(Key, location, '广州')
    res = requests.get(url).json()
    if res['status'] == '0':
        dict_location[i] = np.nan
        counter +=1
    else:
        try: 
            dict_location[i] = res['geocodes'][0]['location']
            if counter % 250 == 0:
                print('one batch(250) processed')
        except:
            dict_location[i] = np.nan
            
        counter +=1

one batch(250) processed


In [4]:
df['location_coords'] = np.nan

counter = 0
for i in list_location:
    counter += 1
    df_temp = df[df['loupandizhi'] == i]
    df_temp_index = df_temp.index.tolist()
    df.iloc[df_temp_index,14] = dict_location[i]
    if counter %50 == 0:
        print('A batch(50) is done')

A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done


In [5]:
df.head(3)

,building_type,dengji,jianzhumianji,loupan_content,loupandizhi,price_perunit,release_date,service_price,suozailouceng,titile,total_price_10K,url,xzl_content,zhuangxiu,location_coords
0,纯写字楼,甲级,118.24㎡,壬丰大厦，坐落在广州天河东路和天河路交界处，周边有学校、科研机构，银行、邮政、酒店等。,"广州, 天河, 石牌桥, 壬丰大厦",(25000元/㎡),2019/7/11 16:00:35,18.00元/平米·月,中区(共44层),岗顶颐高数码楼上，壬丰大厦，118平方低价出售！！,296,https://gz.office.fang.com/shou/3_240443377.html,1) 楼 盘：壬丰大厦 2) 位 置：岗顶颐高数码楼上 3) 面 积： 118.24平方 7...,精装修(不可分割),"113.334937,23.132815"
1,纯写字楼,甲级,110㎡,富邦中心公交地铁网络完善。\r\n\r\n富邦为住宅、写字楼、服务式公寓、商场集一体的综合项...,"广州, 荔湾, 陈家祠, 富邦中心",(18000元/㎡),2019/8/13 10:41:57,15.00元/平米·月,中区(共26层),中山七路 富邦中心 两套小面积 打包出售总110方 单价一万八每平,198,https://gz.office.fang.com/shou/3_241382781.html,中山七路 富邦中心 两套小面积 打包出售总110方 单价一万八每平 两套单位面积分别是53 ...,简装修(不可分割),"113.253790,23.125174"
2,毛坯(不可分割),NaN,140㎡,"富力盈通大厦项目位于珠江新城华夏路，地处中轴线区位,毗邻城市交通动脉黄埔大道，北邻体育西成熟...","广州, 天河, 珠江新城中, 富力盈通大厦",(37000元/㎡),2019/8/20 11:56:51,NaN,中区(共45层),珠江新城 富力盈通大厦 高大上 配套完善,518,https://gz.office.fang.com/shou/3_241578740.html,NaN,NaN,"113.321956,23.126051"


In [10]:
df.shape

(2818, 15)

In [9]:
df.to_csv('广州写字楼地址数据.csv', encoding = 'utf-8-sig',index = False)